# Visualization of networks
The visualization is based on the precision matrices

### Imports

In [ ]:
import os
import sys
import re
import glob
import ast
import os.path as op
import pickle
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx
import scipy as sp

from copy import deepcopy
import copy
import matplotlib.pyplot as plt

import utils

## Load data

In [ ]:
dataset = 'train'
test = False if dataset == 'train' else False

Loading training data

In [ ]:
results_ern_df = pd.read_pickle(f"../data/models_pickles_new_dass/ern_models_{dataset}.pkl")
results_ern_lat_demo_df = pd.read_pickle(f"../data/models_pickles_new_dass/ern_cov_fal_models_{dataset}.pkl")
results_crn_df = pd.read_pickle(f"../data/models_pickles_new_dass/crn_models_{dataset}.pkl")
results_crn_lat_demo_df = pd.read_pickle(f"../data/models_pickles_new_dass/crn_cov_fal2_models_{dataset}.pkl")

train_datasets = [results_ern_df, results_ern_lat_demo_df, results_crn_df, results_crn_lat_demo_df]

Loading testing data

In [ ]:
results_ern_test_df = pd.read_pickle(f"../data/models_pickles_new_dass/ern_models_test.pkl")
results_ern_lat_demo_test_df = pd.read_pickle(f"../data/models_pickles_new_dass/ern_cov_fal_models_test.pkl")
results_crn_test_df = pd.read_pickle(f"../data/models_pickles_new_dass/crn_models_test.pkl")
results_crn_lat_demo_test_df = pd.read_pickle(f"../data/models_pickles_new_dass/crn_cov_fal2_models_test.pkl")

# fill nan
results_crn_lat_demo_test_df['e_LT_F2_C'] = results_crn_lat_demo_test_df['e_LT_F2_C'].fillna(results_crn_lat_demo_test_df['e_LT_F2_C'].mean())

test_datasets = [results_ern_test_df, results_ern_lat_demo_test_df, results_crn_test_df, results_crn_lat_demo_test_df]

Loading precision matrixes

In [ ]:
ern_model_df = pd.read_pickle(f'../data/precision_matrixes/{dataset}/precision_matrix_0_ntrials_1000_sklearn_cv3_th0.65_without_skewd.pkl')
ern_cov_model_df = pd.read_pickle(f'../data/precision_matrixes/{dataset}/precision_matrix_1_ntrials_1000_sklearn_cv3_th0.65_without_skewd_no_sex.pkl')
crn_model_df = pd.read_pickle(f'../data/precision_matrixes/{dataset}/precision_matrix_2_ntrials_1000_sklearn_cv3_th0.65_without_skewd.pkl')
crn_cov_model_df = pd.read_pickle(f'../data/precision_matrixes/{dataset}/precision_matrix_3_ntrials_1000_sklearn_cv3_th0.65_without_skewd_no_sex.pkl')

precision_matrixes = [
    ern_model_df,
    ern_cov_model_df,
    crn_model_df,
    crn_cov_model_df,
]

### Visualize precision matrixes

In [ ]:
mapping = {
    "RRQ": "1Q",
    "DASS-21 Stress": "2Q",
    "DASS-21 Dep": "3Q",
    "STAI-T": "4Q",
    "STAI-S Diff": '5Q',
    "BIS": "6Q",
    "OBSESS": "7Q",
    "HOARD": "8Q",
    "ORD": "9Q",
    "CHECK": "10Q",
    "WBSI": "11Q",
    "IUS-P": "12Q",
    "IUS-I": "13Q",
    "SES": "14Q",
    'BAS_D': "15Q",
    'BAS_PRZY': "16Q",
    'BAS_NAG': "17Q",
    'INDEC_F': "18Q",
    'PUN': "19Q",
    'REW': "20Q",
    'HARM': "21Q",
    'T-CTR': "22Q",
    "OT": "23Q",
    'OB_PERF': "24Q",
    'PS': "25Q",
    'G_SE': "26Q",
    'AMB': "27Q",
    'PRED': "28Q",
    'STAND': "29Q",   
    "Age": "1C",
    "Handness": "2C",
    'e_ERN': "1E",
    'e_LT_F': "4C",
    'performance': "3C",
    'e_CRN': "2E",
    'e_LT_F2_C': "4C",
    'performance': "3C",
}

ern_order = ['1E', '1Q', '2Q', '3Q', '4Q', '5Q', '6Q', '7Q', '8Q', '9Q', '10Q', '11Q', '12Q', '13Q', '14Q', '15Q', '16Q', '17Q', '18Q', '19Q', '20Q', '21Q', '22Q', '23Q', '24Q', '25Q', '26Q', '27Q', '28Q', '29Q',]
ern_cov_order = ['1E', '1Q', '2Q', '3Q', '4Q', '5Q', '6Q', '7Q', '8Q', '9Q', '10Q', '11Q', '12Q', '13Q', '14Q', '15Q', '16Q', '17Q', '18Q', '19Q', '20Q', '21Q', '22Q', '23Q', '24Q', '25Q', '26Q', '27Q', '28Q', '29Q', '1C', '2C', '3C', '4C']
crn_order = ['2E', '1Q', '2Q', '3Q', '4Q', '5Q', '6Q', '7Q', '8Q', '9Q', '10Q', '11Q', '12Q', '13Q', '14Q', '15Q', '16Q', '17Q', '18Q', '19Q', '20Q', '21Q', '22Q', '23Q', '24Q', '25Q', '26Q', '27Q', '28Q', '29Q',]
crn_cov_order = ['2E', '1Q', '2Q', '3Q', '4Q', '5Q', '6Q', '7Q', '8Q', '9Q', '10Q', '11Q', '12Q', '13Q', '14Q', '15Q', '16Q', '17Q', '18Q', '19Q', '20Q', '21Q', '22Q', '23Q', '24Q', '25Q', '26Q', '27Q', '28Q', '29Q', '1C', '2C', '3C', '4C']

orders = [ern_order, ern_cov_order, crn_order, crn_cov_order]

In [ ]:
sns.set(rc={'figure.figsize':(30 , 30)})

for i, (matrix, order) in enumerate(zip(precision_matrixes, orders)):
    plt.figure(i)
    this_matrix = matrix.rename(columns=mapping, index=mapping).reindex(columns=order, index=order)
    this_matrix.to_csv(f'../data/precision_matrixes/precision_matrix_{i}.csv')
    sns.heatmap(
        data = this_matrix,
        annot=True        
    )

### Visualize partial correlations
Based on the precision matrices

In [ ]:
def get_partial_correlations(precision_matrix):
    """
    Computes partial correlations based on the inverse covariance estimates.
    
    Parameters
    ----------
    precision_matrix : pd.DataFrame
        A square DataFrame that represents inverse covariance matrix. Columns and indexes are node names.
    
    Returns
    -------
    partial_correlation_matrix : pd.DataFrame
        A square DataFrame that represents partial correlations matrix. Columns and indexes are node names.

    """
    partial_correlation_matrix = precision_matrix.copy()
    d = 1 / np.sqrt(np.diag(partial_correlation_matrix))
    partial_correlation_matrix *= d
    partial_correlation_matrix *= d[:, np.newaxis]
    partial_correlation_matrix = -partial_correlation_matrix

    return partial_correlation_matrix

In [ ]:
partial_correlation_matrices = [get_partial_correlations(precision_matrix) for precision_matrix in precision_matrixes]

In [ ]:
sns.set(rc={'figure.figsize':(30 , 30)})

for i, (matrix, order) in enumerate(zip(partial_correlation_matrices, orders)):
    plt.figure(i)
    this_matrix = matrix.rename(columns=mapping, index=mapping).reindex(columns=order, index=order)
    this_matrix.to_csv(f'../data/precision_matrixes/partial_correlation_matrix_{i}.csv')
    
    mask = np.triu(this_matrix)
    sns.heatmap(
        data = this_matrix,
        annot=True,
        center = 0.0,
        cmap='vlag',
        mask=mask,
    )

## Visualize ERN and CRN graphs

In [ ]:
save_graphs = False

### 1. Extract list of links from precision matrices

In [ ]:
if test:
    test_links = []

    for matrix in precision_matrixes_test:
        this_links = utils.get_links(matrix)
        test_links.append(this_links)
else:
    links = []

    for matrix in precision_matrixes:
        this_links = utils.get_links(matrix)
        links.append(this_links)

### 2. Calculate within- and out-of-sample nodes predictability

In [ ]:
within_sample_predictability = [
    utils.calculate_nodes_predictability(X, precision_matrix) 
    for X, precision_matrix in zip(train_datasets, precision_matrixes)
]

out_of_sample_predictability = [
    utils.calculate_nodes_predictability(X, precision_matrix) 
    for X, precision_matrix in zip(test_datasets, precision_matrixes)
]

### 3. Bulid the graph from the links

Node labels mappers for plotting

In [ ]:
mapping = {
    "RRQ": "1Q",
    "DASS-21 Stress": "2Q",
    "DASS-21 Dep": "3Q",
    "STAI-T": "4Q",
    "STAI-S Diff": '5Q',
    "BIS": "6Q",
    "OBSESS": "7Q",
    "HOARD": "8Q",
    "ORD": "9Q",
    "CHECK": "10Q",
    "WBSI": "11Q",
    "IUS-P": "12Q",
    "IUS-I": "13Q",
    "SES": "14Q",
    'BAS_D': "15Q",
    'BAS_PRZY': "16Q",
    'BAS_NAG': "17Q",
    'INDEC_F': "18Q",
    'PUN': "19Q",
    'REW': "20Q",
    'HARM': "21Q",
    'T-CTR': "22Q",
    "OT": "23Q",
    'OB_PERF': "24Q",
    'PS': "25Q",
    'G_SE': "26Q",
    'AMB': "27Q",
    'PRED': "28Q",
    'STAND': "29Q",   
    "Age": "1C",
    "Handness": "2C",
    'e_ERN': "1E",
    'e_LT_F': "4C",
    'performance': "3C",
    'e_CRN': "2E",
    'e_LT_F2_C': "4C",
}

#### ERN cov graph

In [ ]:
positions = [
    ('3C', [0.2, 0.75]),
]

In [ ]:
ern_cov_layout, ern_cov_G = utils.draw_graph(
    links[1], 
    seed=3, 
    basic_links=utils.leave_n_edges_from_eeg(links[1], N=0),
    basic='first', 
    nodes_predictabilities = out_of_sample_predictability[1], 
    specific_positions=positions,
    mapping=mapping, 
    save = 'ern_cov_net_test_pred_layout' if save_graphs else None
) 

Display predictability of nodes

In [ ]:
with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(within_sample_predictability[1])

with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(out_of_sample_predictability[1])

Save graph

In [ ]:
nx.write_gml(ern_cov_G, '../data/graphs/ern_cov_graph_test_pred_layout.gml')

#### ERN graph

In [ ]:
# GENERATE THE COV GRAPH LAYOUT FIRST!
layout = ern_cov_layout.copy()

# remove covariates nodes from the layout
layout.pop('1C', None)
layout.pop('2C', None)
layout.pop('3C', None)
layout.pop('4C', None)

In [ ]:
ern_layout, ern_G = utils.draw_graph(
    links[0], 
    seed=13, 
    basic_links=utils.leave_n_edges_from_eeg(links[0], N=0),
    basic='first', 
    nodes_predictabilities = out_of_sample_predictability[0], 
    mapping=mapping,     
    layout = layout,
    save='ern_net_test_pred_layout' if save_graphs else None
) 

Display the predictability of nodes

In [ ]:
with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(within_sample_predictability[0])

with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(out_of_sample_predictability[0])

Save graph

In [ ]:
nx.write_gml(ern_G, '../data/graphs/ern_graph_new_layout_test_predictability.gml')

#### CRN graph

In [ ]:
crn_layout = ern_layout.copy()
# assing the position of ERN from the layout to the CRN
crn_layout['2E'] = crn_layout['1E']

# remove ERN node from the layout
crn_layout.pop('1E', None)

In [ ]:
_, crn_G = utils.draw_graph(
    links[2], 
    seed=16, 
    basic_links=utils.leave_n_edges_from_eeg(links[2], N=0), 
    basic='first', 
    mapping=mapping, 
    nodes_predictabilities = out_of_sample_predictability[2],
    specific_positions=positions,
    layout=crn_layout,
    save = 'crn_net_test_pos_layout' if save_graphs else None
) 

Display predictability

In [ ]:
with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(within_sample_predictability[2])

with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(out_of_sample_predictability[2])

Save graph

In [ ]:
nx.write_gml(crn_G, '../data/graphs/crn_graph_test_pred_layout.gml')

#### CRN cov graph

In [ ]:
crn_cov_layout = ern_cov_layout.copy()

# assing the position of ERN from the layout to the CRN
crn_cov_layout['2E'] = crn_cov_layout['1E']

# remove ERN node from the layout
crn_cov_layout.pop('1E', None)

In [ ]:
_, crn_cov_G = utils.draw_graph(
    links[3], 
    seed=8, 
    basic_links=utils.leave_n_edges_from_eeg(links[3], N=0), 
    basic='first', 
    mapping=mapping,
    nodes_predictabilities = out_of_sample_predictability[3],
    specific_positions = positions,
    layout = crn_cov_layout,
    save = 'crn_cov_net_test_pred_layout' if save_graphs else None
) 

Display predictability of nodes

In [ ]:
with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(within_sample_predictability[3])

with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(out_of_sample_predictability[3])

Save graph

In [ ]:
nx.write_gml(crn_cov_G, '../data/graphs/crn_cov_graph_test_pred_layout.gml')

## Visualize ERN and CRN graphs difference

Define node labels mappers

In [ ]:
mapping_difference_graphs = {
    "RRQ": "1Q",
    "DASS-21 Stress": "2Q",
    "DASS-21 Dep": "3Q",
    "STAI-T": "4Q",
    "STAI-S Diff": '5Q',
    "BIS": "6Q",
    "OBSESS": "7Q",
    "HOARD": "8Q",
    "ORD": "9Q",
    "CHECK": "10Q",
    "WBSI": "11Q",
    "IUS-P": "12Q",
    "IUS-I": "13Q",
    "SES": "14Q",
    'BAS_D': "15Q",
    'BAS_PRZY': "16Q",
    'BAS_NAG': "17Q",
    'INDEC_F': "18Q",
    'PUN': "19Q",
    'REW': "20Q",
    'HARM': "21Q",
    'T-CTR': "22Q",
    "OT": "23Q",
    'OB_PERF': "24Q",
    'PS': "25Q",
    'G_SE': "26Q",
    'AMB': "27Q",
    'PRED': "28Q",
    'STAND': "29Q",   
    "Age": "1C",
    "Handness": "2C",
    'e_ERN': "1E",
    'e_LT_F': "4C",
    'performance': "3C",
    'e_CRN': "1E",
    'e_LT_F2_C': "4C",
}

# mapping_crn = {
#     "RRQ": "1Q",
#     "DASS-21 Stress": "2Q",
#     "DASS-21 Dep": "3Q",
#     "STAI-T": "4Q",
#     "STAI-S Diff": '5Q',
#     "BIS": "6Q",
#     "OBSESS": "7Q",
#     "HOARD": "8Q",
#     "ORD": "9Q",
#     "CHECK": "10Q",
#     "WBSI": "11Q",
#     "IUS-P": "12Q",
#     "IUS-I": "13Q",
#     "SES": "14Q",
#     'BAS_D': "15Q",
#     'BAS_PRZY': "16Q",
#     'BAS_NAG': "17Q",
#     'INDEC_F': "18Q",
#     'PUN': "19Q",
#     'REW': "20Q",
#     'HARM': "21Q",
#     'T-CTR': "22Q",
#     "OT": "23Q",
#     'OB_PERF': "24Q",
#     'PS': "25Q",
#     'G_SE': "26Q",
#     'AMB': "27Q",
#     'PRED': "28Q",
#     'STAND': "29Q",   
#     "Age": "1C",
#     "Handness": "2C",
#     'e_CRN': "1E",
#     'e_LT_F2_C': "4C",
#     'performance': "3C",
# }

### Create graphs from precision matrices

Here, the threshold is set very low so as not to introduce an artificial qualitative difference between ERN and CRN networks when the link is slightly below the threshold in one network (and will be zeroed) and slightly above in the second (and will not be zeroed).

In [ ]:
def create_graph(links, mapping):
    G = nx.from_pandas_edgelist(links,'var1','var2', edge_attr='weight', create_using=nx.Graph())
    G = nx.relabel_nodes(G, mapping)
    return G

In [ ]:
links_for_diffs = [utils.leave_n_edges_from_eeg(utils.get_links(matrix_, threshold=0.005), N=2) for matrix_ in precision_matrixes]
graphs_for_diffs = [create_graph(links, mapping_difference_graphs) for links in links_for_diffs]

### Estimate difference graphs

#### 1. ERN minus CRN model

In [ ]:
difference_graph_ern, colors, styles = utils.estimate_difference_graph(
    graphs_for_diffs[0], 
    graphs_for_diffs[2],
    threshold=0.02
)

utils.draw_difference_graph(
    difference_graph_ern, 
    colors=colors, 
    linestyle=styles, 
    seed=16, 
    no_purple=True, 
    scale=1.2,
    layout = ern_layout,
    save = 'ern_crn_diff_layout' if save_graphs else None
)

Calculate nodes predictability

In [ ]:
G_ = difference_graph_ern.copy()       
ern_crn_diff_matrix = nx.to_numpy_array(G_)
ern_crn_diff_matrix_df = pd.DataFrame(ern_crn_diff_matrix, columns=G_.nodes(), index=G_.nodes())

In [ ]:
ern_crn_diff_within_sample_predictability = utils.calculate_nodes_predictability(train_datasets[0].rename(columns=mapping_difference_graphs), ern_crn_diff_matrix_df)

with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(ern_crn_diff_within_sample_predictability)

ern_crn_diff_out_of_sample_predictability = utils.calculate_nodes_predictability(test_datasets[0].rename(columns=mapping_difference_graphs), ern_crn_diff_matrix_df)

with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(ern_crn_diff_out_of_sample_predictability)

Save the graph

In [ ]:
nx.write_gml(difference_graph_ern, '../data/graphs/ern_crn_diff_graph_layout.gml')
ern_crn_diff_matrix_df.to_pickle(f'../data/precision_matrixes/{dataset}/precision_matrix_ern_crn_diff_layout.pkl')

#### ERN with covariates minus CRN with covariates

In [ ]:
difference_graph_ern_cov, colors, styles = utils.estimate_difference_graph(
    graphs_for_diffs[1], 
    graphs_for_diffs[3],
    threshold=0.02
)

utils.draw_difference_graph(
    difference_graph_ern_cov, 
    colors=colors, 
    linestyle=styles, 
    seed=7, 
    no_purple=True,
    scale=1.3,
    layout = ern_cov_layout,
    save = 'ern_crn_cov_diff_layout' if save_graphs else None
)

Calculate nodes predictability

In [ ]:
G_ = difference_graph_ern_cov.copy()       
ern_crn_cov_diff_matrix = nx.to_numpy_array(G_)
ern_crn_cov_diff_matrix_df = pd.DataFrame(ern_crn_cov_diff_matrix, columns=G_.nodes(), index=G_.nodes())

In [ ]:
ern_crn_cov_diff_within_sample_predictability = utils.calculate_nodes_predictability(train_datasets[1].rename(columns=mapping_difference_graphs), ern_crn_cov_diff_matrix_df)

with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(ern_crn_cov_diff_within_sample_predictability)

ern_crn_cov_diff_out_of_sample_predictability = utils.calculate_nodes_predictability(test_datasets[1].rename(columns=mapping_difference_graphs), ern_crn_cov_diff_matrix_df)

with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(ern_crn_cov_diff_out_of_sample_predictability)

Save the graph

In [ ]:
nx.write_gml(difference_graph_ern_cov, '../data/graphs/ern_crn_cov_diff_graph_layout.gml')
ern_crn_cov_diff_matrix_df.to_pickle(f'../data/precision_matrixes/{dataset}/precision_matrix_ern_crn_cov_diff_layout.pkl')

---
## Draw legend

- ERN

In [ ]:
legend_labels = {
    "1Q: Rumination": "1Q",
    "2Q: Stress": "2Q",
    "3Q: Depression": "3Q",
    "4Q: Anxiety trait": "4Q",
    "5Q: Affective load": '5Q',
    "6Q: Behavioral inhibition": "6Q",
    "7Q: Obsessing": "7Q",
    "8Q: Hoarding": "8Q",
    "9Q: Ordering": "9Q",
    "10Q: Checking": "10Q",
    "11Q: Thought supression": "11Q",
    "12Q: Prospective intolerance of uncertainty": "12Q",
    "13Q: Inhibitory intolerance of uncertainty": "13Q",
    "14Q: Self-esteem": "14Q",
    '15Q: Drive': "15Q",
    '16Q: Fun-seeking': "16Q",
    '17Q: Reward responsiveness': "17Q",
    '18Q: Indecisivness': "18Q",
    '19Q: Punishment sensitivity': "19Q",
    '20Q: Reward sensitivity': "20Q",
    '21Q: Inflated harm responsibility': "21Q",
    '22Q: Importance of thought control': "22Q",
    "23Q: Threat overestimation": "23Q",
    '24Q: Perfectionism/intolerance of uncertainty': "24Q",
    '25Q: Personal standards': "25Q",
    '26Q: Guilt sensitivity': "26Q",
    '27Q: Avoidance of ambiguity': "27Q",
    '28Q: Need for predictability': "28Q",
    '29Q: High standards': "29Q",   
    "1C: Age": "1C",
    "2C: Handedness": "2C",
    '3C: Performance': "3C",
    '4C: ERN latency': "4C",
    '1E: ERN amplitude': "1E",
    # '2E: CRN': "2E",
}

In [ ]:
cm = 1/2.54
dpi = 500

plt.rcParams['figure.dpi'] = dpi
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams['ytick.labelsize'] = 6
plt.rcParams['xtick.labelsize'] = 6
plt.rcParams['axes.labelsize'] = 6
plt.rcParams["axes.edgecolor"] = ".15"
plt.rcParams["axes.linewidth"]  = 0.5
sns.set_style("white")


palette = sns.color_palette("colorblind")

fig = plt.figure(3, figsize=(10*cm, 10*cm))
ax = fig.add_axes([0,0,1,1], aspect=1)
ax.spines[['right', 'top', 'left', 'bottom']].set_visible(False)
# ax = plt.gca()
ax.axes.xaxis.set_visible(False)
ax.axes.yaxis.set_visible(False)
    
# # change default range so that new circles will work
# ax.set_xlim((0, 10))
# ax.set_ylim((0, 10))
# some data
x = [0.5]*len(legend_labels)
y = np.linspace(0,1,len(legend_labels))

nodes_color=[palette[0] if item[1].find('Q') != -1 
                else palette[1]if item[1].find('E') != -1 
                else palette[2] for item in legend_labels.items()]


    
# ax.plot(x, y, 'o', color=nodes_color,  markersize=4)

for i, item in enumerate(legend_labels.items()):
    item_index = len(legend_labels) - i - 1
    
    ax.plot(
        x[item_index],
        y[item_index], 
        'o', 
        markersize=5,
        color=nodes_color[i],
    )
    
    plt.annotate(
        item[0],                       # label text
        (x[item_index], y[item_index]),                    # (x, y)
        textcoords="offset points",   
        xytext=(5, -1.35),               # (x, y) offset amount
        ha='left',
        fontsize=5
    )
    
plt.show()

fig.savefig('../images/ern_legend.png')

- CRN

In [ ]:
legend_labels = {
    "1Q: Rumination": "1Q",
    "2Q: Stress": "2Q",
    "3Q: Depression": "3Q",
    "4Q: Anxiety trait": "4Q",
    "5Q: Affective load": '5Q',
    "6Q: Behavioral inhibition": "6Q",
    "7Q: Obsessing": "7Q",
    "8Q: Hoarding": "8Q",
    "9Q: Ordering": "9Q",
    "10Q: Checking": "10Q",
    "11Q: Thought supression": "11Q",
    "12Q: Prospective intolerance of uncertainty": "12Q",
    "13Q: Inhibitory intolerance of uncertainty": "13Q",
    "14Q: Self-esteem": "14Q",
    '15Q: Drive': "15Q",
    '16Q: Fun-seeking': "16Q",
    '17Q: Reward responsiveness': "17Q",
    '18Q: Indecisivness': "18Q",
    '19Q: Punishment sensitivity': "19Q",
    '20Q: Reward sensitivity': "20Q",
    '21Q: Inflated harm responsibility': "21Q",
    '22Q: Importance of thought control': "22Q",
    "23Q: Threat overestimation": "23Q",
    '24Q: Perfectionism/intolerance of uncertainty': "24Q",
    '25Q: Personal standards': "25Q",
    '26Q: Guilt sensitivity': "26Q",
    '27Q: Avoidance of ambiguity': "27Q",
    '28Q: Need for predictability': "28Q",
    '29Q: High standards': "29Q",   
    "1C: Age": "1C",
    "2C: Handedness": "2C",
    '3C: Performance': "3C",
    '4C: CRN latency': "4C",
    '2E: CRN amplitude': "1E",
    # '2E: CRN': "2E",
}

In [ ]:
cm = 1/2.54
dpi = 500

plt.rcParams['figure.dpi'] = dpi
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams['ytick.labelsize'] = 6
plt.rcParams['xtick.labelsize'] = 6
plt.rcParams['axes.labelsize'] = 6
plt.rcParams["axes.edgecolor"] = ".15"
plt.rcParams["axes.linewidth"]  = 0.5
sns.set_style("white")


palette = sns.color_palette("colorblind")

fig = plt.figure(3, figsize=(10*cm, 10*cm))
ax = fig.add_axes([0,0,1,1], aspect=1)
ax.spines[['right', 'top', 'left', 'bottom']].set_visible(False)
# ax = plt.gca()
ax.axes.xaxis.set_visible(False)
ax.axes.yaxis.set_visible(False)
    
# # change default range so that new circles will work
# ax.set_xlim((0, 10))
# ax.set_ylim((0, 10))
# some data
x = [0.5]*len(legend_labels)
y = np.linspace(0,1,len(legend_labels))

nodes_color=[palette[0] if item[1].find('Q') != -1 
                else palette[1]if item[1].find('E') != -1 
                else palette[2] for item in legend_labels.items()]


    
# ax.plot(x, y, 'o', color=nodes_color,  markersize=4)

for i, item in enumerate(legend_labels.items()):
    item_index = len(legend_labels) - i - 1
    
    ax.plot(
        x[item_index],
        y[item_index], 
        'o', 
        markersize=5,
        color=nodes_color[i],
    )
    
    plt.annotate(
        item[0],                       # label text
        (x[item_index], y[item_index]),                    # (x, y)
        textcoords="offset points",   
        xytext=(5, -1.35),               # (x, y) offset amount
        ha='left',
        fontsize=5
    )
    
plt.show()

fig.savefig('../images/crn_legend.png')

- Diff

In [ ]:
legend_labels = {
    "1Q: Rumination": "1Q",
    "2Q: Stress": "2Q",
    "3Q: Depression": "3Q",
    "4Q: Anxiety trait": "4Q",
    "5Q: Affective load": '5Q',
    "6Q: Behavioral inhibition": "6Q",
    "7Q: Obsessing": "7Q",
    "8Q: Hoarding": "8Q",
    "9Q: Ordering": "9Q",
    "10Q: Checking": "10Q",
    "11Q: Thought supression": "11Q",
    "12Q: Prospective intolerance of uncertainty": "12Q",
    "13Q: Inhibitory intolerance of uncertainty": "13Q",
    "14Q: Self-esteem": "14Q",
    '15Q: Drive': "15Q",
    '16Q: Fun-seeking': "16Q",
    '17Q: Reward responsiveness': "17Q",
    '18Q: Indecisivness': "18Q",
    '19Q: Punishment sensitivity': "19Q",
    '20Q: Reward sensitivity': "20Q",
    '21Q: Inflated harm responsibility': "21Q",
    '22Q: Importance of thought control': "22Q",
    "23Q: Threat overestimation": "23Q",
    '24Q: Perfectionism/intolerance of uncertainty': "24Q",
    '25Q: Personal standards': "25Q",
    '26Q: Guilt sensitivity': "26Q",
    '27Q: Avoidance of ambiguity': "27Q",
    '28Q: Need for predictability': "28Q",
    '29Q: High standards': "29Q",   
    "1C: Age": "1C",
    "2C: Handedness": "2C",
    '3C: Performance': "3C",
    '4C: ERP latency': "4C",
    '2E: ERN amplitude': "1E",
    # '2E: CRN': "2E",
}

In [ ]:
cm = 1/2.54
dpi = 500

plt.rcParams['figure.dpi'] = dpi
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams['ytick.labelsize'] = 6
plt.rcParams['xtick.labelsize'] = 6
plt.rcParams['axes.labelsize'] = 6
plt.rcParams["axes.edgecolor"] = ".15"
plt.rcParams["axes.linewidth"]  = 0.5
sns.set_style("white")


palette = sns.color_palette("colorblind")

fig = plt.figure(3, figsize=(10*cm, 10*cm))
ax = fig.add_axes([0,0,1,1], aspect=1)
ax.spines[['right', 'top', 'left', 'bottom']].set_visible(False)
# ax = plt.gca()
ax.axes.xaxis.set_visible(False)
ax.axes.yaxis.set_visible(False)
    
# # change default range so that new circles will work
# ax.set_xlim((0, 10))
# ax.set_ylim((0, 10))
# some data
x = [0.5]*len(legend_labels)
y = np.linspace(0,1,len(legend_labels))

nodes_color=[palette[0] if item[1].find('Q') != -1 
                else palette[1]if item[1].find('E') != -1 
                else palette[2] for item in legend_labels.items()]


    
# ax.plot(x, y, 'o', color=nodes_color,  markersize=4)

for i, item in enumerate(legend_labels.items()):
    item_index = len(legend_labels) - i - 1
    
    ax.plot(
        x[item_index],
        y[item_index], 
        'o', 
        markersize=5,
        color=nodes_color[i],
    )
    
    plt.annotate(
        item[0],                       # label text
        (x[item_index], y[item_index]),                    # (x, y)
        textcoords="offset points",   
        xytext=(5, -1.35),               # (x, y) offset amount
        ha='left',
        fontsize=5
    )
    
plt.show()

fig.savefig('../images/diff_legend.png')